In [2]:
import pandas as pd
import numpy as np
import random
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 2) Autoregressive SARIMAX 예측
# =========================================================
def autoregressive_sarimax(train, test, exog_cols):

    history = train.copy()
    preds = []

    for i in range(len(test)):

        exog_row = test.loc[[i], exog_cols]
        exog_row = exog_row.fillna(0)

        model = SARIMAX(
            history['일매출'],
            exog=history[exog_cols],
            order=(1,1,1),
            seasonal_order=(1,1,1,7)
        )

        fit = model.fit(disp=False)

        forecast = fit.predict(
            start=len(history),
            end=len(history),
            exog=exog_row
        )

        pred = float(forecast.values[0])
        pred = max(pred, 0)

        preds.append(pred)

        new_row = test.iloc[i].copy()
        new_row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return preds

# =========================================================
# 3) 데이터 로드
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test  = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test ['영업일자'] = pd.to_datetime(test ['영업일자'])

train['일매출'] = train['일매출'].astype(str).str.replace(",", "").astype(float)
test ['일매출'] = test ['일매출'].astype(str).str.replace(",", "").astype(float)

# =========================================================
# 4) 학사일정 merge
# =========================================================
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')

academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test  = test.merge(academic, on='영업일자', how='left')

# =========================================================
# 5) 날짜 기반 피처 추가
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

train = add_date_features(train)
test  = add_date_features(test)

# =========================================================
# 6) SARIMAX exog feature 선택 (lag/rolling 제거)
# =========================================================
exog_cols = [
    'acad_weekend','acad_semester','acad_weekday',
    'acad_holiday','acad_seasonal','acad_exam',
    'acad_ceremony','acad_dormitory',
    'month','weekday','is_weekend','open_hours'
]

train[exog_cols] = train[exog_cols].fillna(0)
test [exog_cols] = test [exog_cols].fillna(0)

# =========================================================
# 7) SARIMAX 예측
# =========================================================
test_pred = autoregressive_sarimax(train, test, exog_cols)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("\n====== SARIMAX 결과 (lag/rolling 제거) ======")
print("MAE  :", mae)
print("RMSE :", rmse)
print("SMAPE:", smape_val)

# =========================================================
# 8) CSV 저장
# =========================================================
output = test[['영업일자']].copy()
output['예측매출'] = test_pred
output.to_csv("sarimax_prediction_output.csv", index=False)

print("\nCSV 파일 저장 완료 → sarimax_prediction_output.csv")
print(output.head())



====== SARIMAX 결과 (lag/rolling 제거) ======
MAE  : 77898.79778632565
RMSE : 98289.41421725308
SMAPE: 76.65960579041068

CSV 파일 저장 완료 → sarimax_prediction_output.csv
        영업일자           예측매출
0 2025-08-10       0.000000
1 2025-08-11  185742.622154
2 2025-08-12  122451.625520
3 2025-08-13  162148.301086
4 2025-08-14  153032.472745


### **하이퍼파라미터 자동 탐색**

In [4]:
import itertools
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ================================
# Seed
# ================================
def set_seed(seed=42):
    np.random.seed(seed)
set_seed(42)

# ================================
# 평가 지표
# ================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    mae  = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smp  = smape(actual, pred)
    return mae, rmse, smp


# =========================================================
# 1) SARIMAX AIC 기반 자동 탐색기
# =========================================================
def sarimax_auto_search(train_df, exog_cols):

    y = train_df["일매출"]
    exog = train_df[exog_cols]

    # 검색 공간 (너무 넓히면 과부하! 실무에서도 이렇게 씀)
    p = d = q = range(0, 2)
    P = D = Q = range(0, 2)
    seasonal_period = [7]   # 요일 패턴

    best_aic = np.inf
    best_param = None
    best_model = None

    search_space = list(itertools.product(p, d, q, P, D, Q, seasonal_period))

    print(f"총 탐색 조합 수: {len(search_space)}\n")

    for (p_, d_, q_, P_, D_, Q_, s_) in search_space:

        try:
            model = SARIMAX(
                y,
                exog=exog,
                order=(p_, d_, q_),
                seasonal_order=(P_, D_, Q_, s_),
                enforce_stationarity=False,
                enforce_invertibility=False
            )

            fit = model.fit(disp=False)

            if fit.aic < best_aic:
                best_aic = fit.aic
                best_param = (p_, d_, q_, P_, D_, Q_, s_)
                best_model = fit

                print(f"🔥 새로운 최적 AIC: {best_aic:.2f} → {best_param}")

        except Exception as e:
            continue

    print("\n==========================")
    print("📌 최종 선택된 SARIMAX 모델")
    print("order =", best_param[:3])
    print("seasonal_order =", best_param[3:])
    print("AIC =", best_aic)
    print("==========================\n")

    return best_model


# =========================================================
# 2) Auto-Regressive 예측
# =========================================================
def sarimax_forecast_autoreg(train_df, test_df, exog_cols, best_model):

    history = train_df.copy()
    preds = []

    # 🔥 최신 statsmodels 버전에서 지원하는 order 추출 방식
    order = best_model.model.order
    seasonal_order = best_model.model.seasonal_order

    for i in range(len(test_df)):

        exog_next = test_df.loc[[i], exog_cols].fillna(0)

        model = SARIMAX(
            history["일매출"],
            exog=history[exog_cols],
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        fit = model.fit(disp=False)

        forecast = fit.predict(
            start=len(history),
            end=len(history),
            exog=exog_next
        )

        pred = float(forecast.values[0])
        pred = max(pred, 0)

        preds.append(pred)

        new_row = test_df.iloc[i].copy()
        new_row["일매출"] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return preds


# =========================================================
# 3) 데이터 로드 (경로 변경 없음)
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test  = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test ['영업일자'] = pd.to_datetime(test ['영업일자'])

train['일매출'] = train['일매출'].astype(str).str.replace(",", "").astype(float)
test ['일매출'] = test ['일매출'].astype(str).str.replace(",", "").astype(float)


# =========================================================
# 4) 학사일정 merge (그대로)
# =========================================================
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test  = test.merge(academic, on='영업일자', how='left')


# =========================================================
# 5) 날짜 피처 추가
# =========================================================
def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

train = add_date_features(train)
test  = add_date_features(test)


# =========================================================
# 6) exog columns (GRU와 동일)
# =========================================================
exog_cols = [
    'acad_weekend','acad_semester','acad_weekday',
    'acad_holiday','acad_seasonal','acad_exam',
    'acad_ceremony','acad_dormitory',
    'month','weekday','is_weekend','open_hours'
]

train[exog_cols] = train[exog_cols].fillna(0)
test [exog_cols] = test [exog_cols].fillna(0)


# =========================================================
# 7) AIC 기반 자동 SARIMAX 모델 선택
# =========================================================
best_model = sarimax_auto_search(train, exog_cols)


# =========================================================
# 8) 예측
# =========================================================
test_pred = sarimax_forecast_autoreg(train, test, exog_cols, best_model)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("====== SARIMAX 자동탐색 결과 ======")
print("MAE  :", mae)
print("RMSE :", rmse)
print("SMAPE:", smape_val)


# =========================================================
# 9) CSV 저장
# =========================================================
output = test[['영업일자']].copy()
output['예측매출'] = test_pred
output.to_csv("sarimax_auto_prediction.csv", index=False)

print("\nCSV 저장 완료 → sarimax_auto_prediction.csv")


총 탐색 조합 수: 64

🔥 새로운 최적 AIC: 23484.44 → (0, 0, 0, 0, 0, 0, 7)
🔥 새로운 최적 AIC: 19090.44 → (0, 0, 0, 0, 0, 1, 7)
🔥 새로운 최적 AIC: 19026.92 → (0, 0, 0, 0, 1, 1, 7)
🔥 새로운 최적 AIC: 19006.22 → (0, 0, 0, 1, 1, 1, 7)
🔥 새로운 최적 AIC: 18990.11 → (0, 0, 1, 0, 1, 1, 7)
🔥 새로운 최적 AIC: 18974.29 → (0, 0, 1, 1, 1, 1, 7)
🔥 새로운 최적 AIC: 18959.89 → (1, 0, 1, 0, 1, 1, 7)
🔥 새로운 최적 AIC: 18952.35 → (1, 0, 1, 1, 1, 1, 7)

📌 최종 선택된 SARIMAX 모델
order = (1, 0, 1)
seasonal_order = (1, 1, 1, 7)
AIC = 18952.35459120816

====== SARIMAX 자동탐색 결과 ======
MAE  : 63429.931170126794
RMSE : 79420.76312946768
SMAPE: 76.60371037543867

CSV 저장 완료 → sarimax_auto_prediction.csv


### **GRU와 SARIMAX 블렌딩**

In [5]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")


# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)



# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v



# =========================================================
# 2) 데이터 전처리
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출'].astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df



# =========================================================
# 3) GRU Dataset
# =========================================================
def create_sequences(df, feature_cols, seq_len=28):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출'].values
    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])
    return np.array(X), np.array(y)


class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



# =========================================================
# 4) GRU 모델 정의
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])



# =========================================================
# 5) GRU Autoregressive 예측
# =========================================================
def autoregressive_forecast(model, df_train, df_test, seq_len, feature_cols, scaler_X, scaler_y):

    history = df_train.copy()
    preds = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()
        last_vals = history['일매출'].values

        row['lag1'] = last_vals[-1]
        row['lag7'] = last_vals[-7]
        row['lag14'] = last_vals[-14]
        row['lag28'] = last_vals[-28]

        row['roll_mean7'] = pd.Series(last_vals[-7:]).mean()
        row['roll_mean14'] = pd.Series(last_vals[-14:]).mean()
        row['roll_mean28'] = pd.Series(last_vals[-28:]).mean()
        row['roll_std7'] = pd.Series(last_vals[-7:]).std()
        row['roll_std28'] = pd.Series(last_vals[-28:]).std()

        seq_df = history.tail(seq_len).copy()
        seq_scaled = scaler_X.transform(seq_df[feature_cols])
        X = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        pred_scaled = model(X).item()
        pred = scaler_y.inverse_transform([[pred_scaled]])[0][0]
        pred = max(pred, 0)

        preds.append(pred)

        row['일매출'] = pred
        history = pd.concat([history, pd.DataFrame([row])], ignore_index=True)

    return preds



# =========================================================
# 6) SARIMAX 예측 (exog 기반)
# =========================================================
def sarimax_forecast(train_df, test_df, exog_cols):

    # SARIMAX 기본 파라미터 (요일 주기)
    order = (1,1,1)
    seasonal = (1,1,1,7)

    history = train_df.copy()
    preds = []

    for i in range(len(test_df)):

        model = SARIMAX(
            history["일매출"],
            exog=history[exog_cols],
            order=order,
            seasonal_order=seasonal,
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        fit = model.fit(disp=False)

        exog_next = test_df.loc[[i], exog_cols].fillna(0)

        forecast = fit.predict(
            start=len(history),
            end=len(history),
            exog=exog_next
        )

        pred = float(forecast.values[0])
        pred = max(pred, 0)

        preds.append(pred)

        new_row = test_df.iloc[i].copy()
        new_row["일매출"] = pred
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)

    return preds



# =========================================================
# 7) 데이터 로드
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자'] = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test = clean_sales(test)



# ----------------- 학사일정 merge -----------------
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test = test.merge(academic, on='영업일자', how='left')

train = add_date_features(train)
test = add_date_features(test)

train = add_lag_features(train).dropna().reset_index(drop=True)
test = add_lag_features(test)



# =========================================================
# 8) GRU 학습
# =========================================================
feature_cols = [
    'acad_weekend','acad_semester','acad_weekday',
    'open_hours','acad_ceremony','acad_exam',
    'lag1','lag7','lag14','lag28',
    'roll_std7','roll_std28',
    'roll_mean7','roll_mean14','roll_mean28'
]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출']] = scaler_y.fit_transform(train[['일매출']])

seq_len = 28
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = GRUModel(input_dim=len(feature_cols), num_layers=2)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/50] Loss: {total_loss:.4f}")



# =========================================================
# 9) GRU + SARIMAX 예측 + Blending
# =========================================================
gru_pred = autoregressive_forecast(model, train, test, seq_len, feature_cols, scaler_X, scaler_y)

# SARIMAX exog subset
exog_cols = [
    'acad_weekend','acad_semester','acad_weekday',
    'acad_holiday','acad_seasonal','acad_exam',
    'acad_ceremony','acad_dormitory',
    'month','weekday','is_weekend','open_hours'
]

sarimax_pred = sarimax_forecast(train, test, exog_cols)

# ---------- Blending ----------
alpha = 0.7     # GRU 가중치
final_pred = alpha * np.array(gru_pred) + (1-alpha) * np.array(sarimax_pred)



# =========================================================
# 10) 평가
# =========================================================
mae, rmse, smape_val = evaluate(test['일매출'], final_pred)
print("\n====== GRU + SARIMAX Blending 결과 ======")
print("MAE   :", mae)
print("RMSE  :", rmse)
print("SMAPE :", smape_val)



# =========================================================
# 11) CSV 저장
# =========================================================
output = test[['영업일자']].copy()
output['GRU'] = gru_pred
output['SARIMAX'] = sarimax_pred
output['Blended'] = final_pred

output.to_csv("gru_sarimax_blended.csv", index=False)
print("\nCSV 저장 완료 → gru_sarimax_blended.csv")


[Epoch 1/50] Loss: 0.8714
[Epoch 2/50] Loss: 0.5253
[Epoch 3/50] Loss: 0.4432
[Epoch 4/50] Loss: 0.3666
[Epoch 5/50] Loss: 0.3487
[Epoch 6/50] Loss: 0.3307
[Epoch 7/50] Loss: 0.3320
[Epoch 8/50] Loss: 0.3193
[Epoch 9/50] Loss: 0.3023
[Epoch 10/50] Loss: 0.2984
[Epoch 11/50] Loss: 0.2922
[Epoch 12/50] Loss: 0.2591
[Epoch 13/50] Loss: 0.2478
[Epoch 14/50] Loss: 0.2435
[Epoch 15/50] Loss: 0.2300
[Epoch 16/50] Loss: 0.2335
[Epoch 17/50] Loss: 0.2289
[Epoch 18/50] Loss: 0.2137
[Epoch 19/50] Loss: 0.2119
[Epoch 20/50] Loss: 0.2062
[Epoch 21/50] Loss: 0.2119
[Epoch 22/50] Loss: 0.2106
[Epoch 23/50] Loss: 0.2130
[Epoch 24/50] Loss: 0.2027
[Epoch 25/50] Loss: 0.2083
[Epoch 26/50] Loss: 0.2082
[Epoch 27/50] Loss: 0.2012
[Epoch 28/50] Loss: 0.1947
[Epoch 29/50] Loss: 0.2009
[Epoch 30/50] Loss: 0.1999
[Epoch 31/50] Loss: 0.2094
[Epoch 32/50] Loss: 0.2004
[Epoch 33/50] Loss: 0.1923
[Epoch 34/50] Loss: 0.1984
[Epoch 35/50] Loss: 0.1918
[Epoch 36/50] Loss: 0.2006
[Epoch 37/50] Loss: 0.1884
[Epoch 38/